In [65]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
from fredapi import Fred

#personal preference
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

file=(open(r'C:\Users\brian\api_keys\fred_api_key.txt','r'))
api_key=file.read()
api_key=api_key[:-1]

# Set up the FRED API
fred_api_key = api_key  # Replace with your FRED API key
fred = Fred(api_key=api_key)

# Define the FRED series IDs for the assets
asset_ids = {
    'Stocks': 'SP500',  # S&P 500 Index
    'Bonds': 'BAMLH0A0HYM2EY',  # ICE BofA US High Yield Index Effective Yield
    'Commercial RealEstate': 'COMREPUSQ159N',  # Commercial Real Estate Prices for United States (Real Estate Proxy)
    'Commodities': 'PPIACO',  # Producer Price Index by Commodity: All Commodities
    'Residential RealEstate': 'QUSR628BIS',  # Residential Real Estate Prices for United States (Real Estate Proxy)
}

# Set the date range for historical data
start_date = dt.datetime(2010, 1, 1)
end_date = dt.datetime.today()

# Retrieve the data from FRED
asset_data = {}
for asset, series_id in asset_ids.items():
    asset_data[asset] = fred.get_series(series_id, start_date, end_date)

# Combine data into a DataFrame
df_assets = pd.DataFrame(asset_data)

# Handle missing data (e.g., filling forward)
df_assets.fillna(method='ffill', inplace=True)

asset_ticker = yf.Ticker("ACWI") #iShares MSCI ACWI ETF
asset = asset_ticker.history(period="max", interval='1mo')
asset=asset["Close"]
asset.index = pd.to_datetime(asset.index)
asset.index=asset.index.date


# Apply pct_change(periods=12) to all assets except RealEstate
df_assets['Stocks'] = df_assets['Stocks'].pct_change(periods=252)
df_assets['Bonds'] = df_assets['Bonds'].pct_change(periods=252)
df_assets['Commodities'] = df_assets['Commodities'].pct_change(periods=12)
df_assets['MSCI All Country World Index (ACWI)']=asset.pct_change(periods=12)
df_assets['Residential RealEstate']=df_assets['Residential RealEstate'].pct_change(periods=12)
df_assets.dropna(inplace=True)


# Calculate mean returns and covariance matrix for Monte Carlo simulation
mean_returns = df_assets.mean().values
corr_matrix = df_assets.corr()

print("Mean Returns:")
print(mean_returns)

Mean Returns:
[1.07471574e-01 6.95500687e-02 3.41205789e+00 2.89131973e-03
 3.13740618e-03 9.75133122e-02]


In [66]:
print("Corellation Matrix:")
corr_matrix

Corellation Matrix:


,Stocks,Bonds,Commercial RealEstate,Commodities,Residential RealEstate,MSCI All Country World Index (ACWI)
Stocks,1.000,-0.799,0.203,0.456,0.155,0.903
Bonds,-0.799,1.000,-0.217,-0.338,-0.228,-0.812
Commercial RealEstate,0.203,-0.217,1.000,0.317,0.186,0.179
Commodities,0.456,-0.338,0.317,1.000,0.143,0.436
Residential RealEstate,0.155,-0.228,0.186,0.143,1.000,0.197
MSCI All Country World Index (ACWI),0.903,-0.812,0.179,0.436,0.197,1.000


In [67]:
df_assets.head()

,Stocks,Bonds,Commercial RealEstate,Commodities,Residential RealEstate,MSCI All Country World Index (ACWI)
2015-10-01,-0.021,0.361,5.805,-0.008,0.019,-0.006
2015-11-01,0.006,0.216,5.805,-0.010,0.000,-0.025
2015-12-01,0.010,0.208,5.805,-0.012,0.000,-0.026
2016-01-01,-0.006,0.319,-0.824,-0.005,0.011,-0.062
2016-02-01,-0.081,0.509,-0.824,-0.007,0.000,-0.122


In [68]:
def simulate_portfolios(mean_returns, corr_matrix, num_portfolios, time_horizon):
    num_assets = len(mean_returns)
    results = np.zeros((num_portfolios, num_assets + 2))  # Store returns, risk, Sharpe Ratio
    
    for i in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)  # Normalize to sum to 1
        
        # Portfolio return and risk calculation
        portfolio_return = np.sum(weights * mean_returns)
        portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(corr_matrix, weights)))
        sharpe_ratio = portfolio_return / portfolio_risk
        
        results[i, :num_assets] = weights
        results[i, num_assets] = portfolio_return
        results[i, num_assets + 1] = portfolio_risk

    return results

# Example of running the simulation
num_portfolios = 10000
time_horizon = 5# in years
simulated_portfolios = simulate_portfolios(mean_returns, corr_matrix, num_portfolios, time_horizon)

# Filter the portfolios for optimization
min_return = 2
filtered_portfolios = [p for p in simulated_portfolios if p[-2] > min_return]

# Select the optimal portfolio
def optimal_portfolio(simulated_portfolios):
    sorted_portfolios = sorted(simulated_portfolios, key=lambda x: x[-1], reverse=True)
    return sorted_portfolios[0]

best_portfolio = optimal_portfolio(filtered_portfolios)

# Print optimal portfolio with asset names and their weights
def print_portfolio(assets, portfolio):
    print("Optimal Portfolio Allocation:")
    for asset, weight in zip(assets, portfolio[:len(assets)]):
        print(f"{asset}: {weight*100:.2f}%")
    print(f"Expected Return: {portfolio[-2]:.2f}")
    print(f"Expected Risk (Standard Deviation): {portfolio[-1]:.2f}")

# Call the function to print the results
print_portfolio(df_assets.keys(), best_portfolio)


Optimal Portfolio Allocation:
Stocks: 9.41%
Bonds: 0.19%
Commercial RealEstate: 64.27%
Commodities: 0.62%
Residential RealEstate: 21.21%
MSCI All Country World Index (ACWI): 4.31%
Expected Return: 2.21
Expected Risk (Standard Deviation): 0.76
